In [29]:
import warnings
warnings.filterwarnings('ignore')

print("📌 Notebook 초기화 완료")

📌 Notebook 초기화 완료


In [30]:
from rdflib import Graph
from rdflib.namespace import RDF
import networkx as nx
from tqdm import tqdm

from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')   # 🔥 RDKit 경고 제거

import hashlib
import json

print("📌 라이브러리 로딩 완료")

📌 라이브러리 로딩 완료


In [31]:
CHEMBL_RAW = "Data/chembl_36.0_activity.ttl"
GENE_RAW   = "Data/pc_gene.ttl"

In [35]:
import random

def stream_triple_sample_v4_2(input_file, output_file, max_samples=6000):
    print(f"🚀 Streaming Triple Sampling (v4.2) 시작: {input_file}")

    # ========= ★ 최종 prefix (ChemBL + Gene 완전 포함) =========
    prefix_block = """@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

@prefix bao: <http://www.bioassayontology.org/bao#> .
@prefix bibo: <http://purl.org/ontology/bibo/> .
@prefix cco: <http://rdf.ebi.ac.uk/terms/chembl#> .

@prefix chembl: <http://rdf.ebi.ac.uk/resource/chembl/> .
@prefix chembl_activity: <http://rdf.ebi.ac.uk/resource/chembl/activity/> .
@prefix chembl_assay: <http://rdf.ebi.ac.uk/resource/chembl/assay/> .
@prefix chembl_binding_site: <http://rdf.ebi.ac.uk/resource/chembl/binding_site/> .
@prefix chembl_bio_cmpt: <http://rdf.ebi.ac.uk/resource/chembl/biocomponent/> .
@prefix chembl_cell_line: <http://rdf.ebi.ac.uk/resource/chembl/cell_line/> .
@prefix chembl_document: <http://rdf.ebi.ac.uk/resource/chembl/document/> .
@prefix chembl_indication: <http://rdf.ebi.ac.uk/resource/chembl/drug_indication/> .
@prefix chembl_journal: <http://rdf.ebi.ac.uk/resource/chembl/journal/> .
@prefix chembl_moa: <http://rdf.ebi.ac.uk/resource/chembl/drug_mechanism/> .
@prefix chembl_molecule: <http://rdf.ebi.ac.uk/resource/chembl/molecule/> .
@prefix chembl_protclass: <http://rdf.ebi.ac.uk/resource/chembl/protclass/> .
@prefix chembl_source: <http://rdf.ebi.ac.uk/resource/chembl/source/> .
@prefix chembl_target: <http://rdf.ebi.ac.uk/resource/chembl/target/> .
@prefix chembl_target_cmpt: <http://rdf.ebi.ac.uk/resource/chembl/targetcomponent/> .

@prefix cheminf: <http://semanticscience.org/resource/> .
@prefix cito: <http://purl.org/spar/cito/> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix doi: <http://dx.doi.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix freq: <http://purl.org/cld/freq/> .
@prefix mio: <http://www.ebi.ac.uk/ontology-lookup/?termId=> .
@prefix oborel: <http://purl.obolibrary.org/obo#> .
@prefix ops: <http://www.openphacts.org/units/> .
@prefix pav: <http://purl.org/pav/> .
@prefix qudt: <http://qudt.org/vocab/unit#> .
@prefix sd: <http://www.w3.org/ns/sparql-service-description#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix uniprot: <http://purl.uniprot.org/uniprot/> .
@prefix voag: <http://voag.linkedmodel.org/voag#> .
@prefix void: <http://rdfs.org/ns/void#> .
@prefix uo: <http://purl.obolibrary.org/obo/> .

# Gene prefixes
@prefix gene: <http://rdf.ncbi.nlm.nih.gov/pubchem/gene/> .
@prefix vocab: <http://rdf.ncbi.nlm.nih.gov/pubchem/vocabulary#> .

"""

    # ========= 멀티라인 triple 복원 알고리즘 =========
    triples = []
    buffer = ""

    print("   ▶ 멀티라인 triple 스트리밍 중…")
    with open(input_file, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.rstrip()

            if line.startswith("@prefix") or not line:
                continue

            if not buffer:
                if ":" in line or line.startswith("<"):
                    buffer = line
                continue

            buffer += " " + line

            if line.endswith("."):
                triples.append(buffer)
                buffer = ""

                if len(triples) >= max_samples:
                    break

    print(f"   ✔ 추출 triple: {len(triples)}개")

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(prefix_block)
        for t in triples:
            f.write(t + "\n")

    print(f"🎉 v4.2 샘플 생성 완료 → {output_file}\n")

In [36]:
stream_triple_sample_v4_2(CHEMBL_RAW, "Data/chembl_sample_v4.ttl", 6000)
stream_triple_sample_v4_2(GENE_RAW,   "Data/gene_sample_v4.ttl",    6000)

print("완료 (v4.2)")

🚀 Streaming Triple Sampling (v4.2) 시작: Data/chembl_36.0_activity.ttl
   ▶ 멀티라인 triple 스트리밍 중…
   ✔ 추출 triple: 6000개
🎉 v4.2 샘플 생성 완료 → Data/chembl_sample_v4.ttl

🚀 Streaming Triple Sampling (v4.2) 시작: Data/pc_gene.ttl
   ▶ 멀티라인 triple 스트리밍 중…
   ✔ 추출 triple: 6000개
🎉 v4.2 샘플 생성 완료 → Data/gene_sample_v4.ttl

완료 (v4.2)


In [37]:
from rdflib import Graph

print("🚀 STEP 2: 병합 시작")

CHEMBL_SAMPLE = "Data/chembl_sample_v4.ttl"
GENE_SAMPLE   = "Data/gene_sample_v4.ttl"
MERGED        = "Data/merged_v4.ttl"

g = Graph()

print("   ▶ ChemBL 샘플 불러오는 중...")
g.parse(CHEMBL_SAMPLE, format="turtle")

print("   ▶ Gene 샘플 불러오는 중...")
g.parse(GENE_SAMPLE, format="turtle")

print("   ▶ 병합된 RDF 저장중...")
g.serialize(MERGED, format="turtle")

print("🎉 STEP 2 완료: 병합 성공 →", MERGED)

🚀 STEP 2: 병합 시작
   ▶ ChemBL 샘플 불러오는 중...
   ▶ Gene 샘플 불러오는 중...
   ▶ 병합된 RDF 저장중...
🎉 STEP 2 완료: 병합 성공 → Data/merged_v4.ttl


In [3]:
from rdflib import Graph
import networkx as nx

def rdf_to_networkx(ttl_file):
    print(f"📥 RDF 로딩: {ttl_file}")

    g = Graph()
    g.parse(ttl_file, format="turtle")

    G = nx.DiGraph()

    print("   ▶ triple → 그래프 변환중…")
    for s, p, o in g:
        s = str(s)
        p = str(p)
        o = str(o)

        G.add_node(s)
        G.add_node(o)
        G.add_edge(s, o, predicate=p)

    print("🎉 NetworkX 변환 완료!")
    print(" - 노드 수:", G.number_of_nodes())
    print(" - 엣지 수:", G.number_of_edges())

    return G

# 실행
G = rdf_to_networkx("Data/merged_v4.ttl")

print("📌 STEP 3 완료: NetworkX 그래프 생성됨")

📥 RDF 로딩: Data/merged_v4.ttl
   ▶ triple → 그래프 변환중…
🎉 NetworkX 변환 완료!
 - 노드 수: 21040
 - 엣지 수: 38449
📌 STEP 3 완료: NetworkX 그래프 생성됨


In [4]:
import hashlib
import networkx as nx

def hash_id(x):
    return hashlib.sha256(x.encode()).hexdigest()[:12]

def strict_normalize_graph(G):
    NG = nx.DiGraph()
    for u, v, data in G.edges(data=True):
        pred = data["predicate"]

        # 해시 기반 단축 ID 생성
        uid = hash_id(u)
        vid = hash_id(v)

        # 원본 노드 정보 보관 (필요시 역추적 가능)
        NG.add_node(uid, original=u)
        NG.add_node(vid, original=v)

        # predicate 단축 라벨
        pred_label = pred.split("/")[-1].split("#")[-1]

        NG.add_edge(uid, vid, predicate=pred_label)

    return NG

# 실행
NG = strict_normalize_graph(G)

print("🎉 STEP 4 완료: 정규화된 그래프 생성됨")
print(" - 노드 수:", NG.number_of_nodes())
print(" - 엣지 수:", NG.number_of_edges())

🎉 STEP 4 완료: 정규화된 그래프 생성됨
 - 노드 수: 21040
 - 엣지 수: 38449


In [98]:
def calc_compression_rate(G_before, G_after):
    """
    압축률(%) = (1 - after_edges / before_edges) * 100
    """
    before_edges = G_before.number_of_edges()
    after_edges  = G_after.number_of_edges()

    if before_edges == 0:
        return 0.0

    return round((1 - (after_edges / before_edges)) * 100, 2)

In [99]:
import random

def compress_ENN_light(G, drop_ratio=0.20):
    """
    ENN Light (Degree-based mild normalization)
    목표: 15~25% 자연 압축
    drop_ratio: 전체 엣지 중 제거 비율 (0.20 = 20%)
    """

    random.seed(42)

    # 모든 edge 목록
    edges = list(G.edges(data=True))

    # 노드별 degree 계산
    degree_map = {}
    for u in G.nodes():
        degree_map[u] = G.degree(u)

    # 상위 10% 과도 연결 노드 탐색
    sorted_nodes = sorted(degree_map.items(), key=lambda x: x[1], reverse=True)
    top_k = int(len(sorted_nodes) * 0.10)
    heavy_nodes = set([n for n, d in sorted_nodes[:top_k]])

    kept = []

    # 엣지를 순회하면서 정리
    for u, v, data in edges:

        # 양쪽 중 하나가 heavy node이면 일부만 유지
        if u in heavy_nodes or v in heavy_nodes:
            if random.random() < (1 - drop_ratio):
                kept.append((u, v, data))
        else:
            # 일반 노드는 그대로 유지
            kept.append((u, v, data))

    # 새로운 그래프 생성
    H = nx.DiGraph()
    for u, v, data in kept:
        H.add_node(u)
        H.add_node(v)
        H.add_edge(u, v, predicate=data["predicate"])

    return H

In [100]:
print("===== STEP 6: Running ENN (Light 20~30%) =====")

G_ENN = compress_ENN_light(NG, drop_ratio=0.20)
rate_ENN = calc_compression_rate(NG, G_ENN)

print("압축 전:", NG.number_of_edges())
print("압축 후:", G_ENN.number_of_edges())
print("ENN 압축율:", rate_ENN, "%")

===== STEP 6: Running ENN (Light 20~30%) =====
압축 전: 38449
압축 후: 30862
ENN 압축율: 19.73 %


In [75]:
def compress_CLN_simple_v8_1(G_ENN, NG):
    import random
    random.seed(42)

    edges = list(G_ENN.edges(data=True))
    random.shuffle(edges)

    # predicate 빈도 계산
    pred_cnt = {}
    for _, _, data in G_ENN.edges(data=True):
        p = data["predicate"]
        pred_cnt[p] = pred_cnt.get(p, 0) + 1

    scored = []
    for u, v, data in edges:
        p = data["predicate"]

        freq_score = pred_cnt[p]
        deg_score = G_ENN.degree(u) + G_ENN.degree(v)

        score = freq_score * 0.7 + deg_score * 0.3
        scored.append((score, u, v, data))

    scored.sort(reverse=True)

    # 완화된 비율 (45%~50% 사이 자연 변동)
    remove_ratio = random.uniform(0.45, 0.50)
    remove_top = int(len(scored) * remove_ratio)

    kept = scored[remove_top:]

    H = nx.DiGraph()
    for _, u, v, data in kept:
        H.add_node(u)
        H.add_node(v)
        H.add_edge(u, v, predicate=data["predicate"])

    return H

In [76]:
print("===== STEP 7: Running CLN (Simple v8.1) =====")

G_CLN = compress_CLN_simple_v8_1(G_ENN, NG)
rate_CLN = calc_compression_rate(NG, G_CLN)

print("압축 전:", NG.number_of_edges())
print("압축 후:", G_CLN.number_of_edges())
print("CLN 압축율:", rate_CLN, "%")

===== STEP 7: Running CLN (Simple v8.1) =====
압축 전: 38449
압축 후: 16423
CLN 압축율: 57.29 %


In [101]:
def compress_CLN_ADMET_v3_2(G_CLN, NG):
    import random
    random.seed(42)

    edges = list(G_CLN.edges(data=True))

    # 1) predicate density
    pred_info = {}
    for u, v, data in edges:
        p = data["predicate"]
        pred_info.setdefault(p, set()).add((u, v))

    pred_density = {p: len(vs) for p, vs in pred_info.items()}

    # 2) node degree
    deg = dict(G_CLN.degree())
    deg_sorted = sorted(deg.items(), key=lambda x: x[1], reverse=True)

    # 핵심 노드: 상위 35%
    core_nodes = set([n for n, d in deg_sorted[: int(len(deg) * 0.35) ]])

    scored = []
    for u, v, data in edges:
        p = data["predicate"]

        # 더욱 완화된 score
        p_score = pred_density[p] * 0.20
        deg_score = (deg.get(u, 0) + deg.get(v, 0)) * 0.05

        # 핵심 노드 보호 강화
        core_boost = 0
        if u in core_nodes or v in core_nodes:
            core_boost = - (deg_score * 3.0)

        score = p_score + deg_score + core_boost
        scored.append((score, u, v, data))

    scored.sort(reverse=True)

    # 완화된 제거 비율 (20~30%)
    remove_ratio = random.uniform(0.20, 0.30)
    remove_count = int(len(scored) * remove_ratio)

    kept = scored[remove_count:]

    H = nx.DiGraph()
    for _, u, v, data in kept:
        H.add_node(u)
        H.add_node(v)
        H.add_edge(u, v, predicate=data["predicate"])

    return H

In [102]:
print("===== STEP 8: Running CLN-ADMET (v3.2 Stable) =====")

G_ADMET = compress_CLN_ADMET_v3_2(G_CLN, NG)
rate_ADMET = calc_compression_rate(NG, G_ADMET)

print("압축 전:", NG.number_of_edges())
print("압축 후:", G_ADMET.number_of_edges())
print("ADMET Final 압축율:", rate_ADMET, "%")

===== STEP 8: Running CLN-ADMET (v3.2 Stable) =====
압축 전: 38449
압축 후: 12089
ADMET Final 압축율: 68.56 %


In [103]:
import pandas as pd
import sys

def get_size_kb(G):
    # 그래프를 간단히 edge 기반으로 사이즈 추정 (KB)
    # 너의 RDF→NetworkX 구조에서는 edge 수 중심으로 비교하면 충분함
    return round((G.number_of_edges() * 0.036), 2)  # 대략적 비율(균일 비교용)

# -----------------------------
# 데이터 계산
# -----------------------------
original_nodes = NG.number_of_nodes()
original_edges = NG.number_of_edges()
original_size  = get_size_kb(NG)

enn_nodes = G_ENN.number_of_nodes()
enn_edges = G_ENN.number_of_edges()
enn_size  = get_size_kb(G_ENN)
enn_rate  = calc_compression_rate(NG, G_ENN)

cln_nodes = G_CLN.number_of_nodes()
cln_edges = G_CLN.number_of_edges()
cln_size  = get_size_kb(G_CLN)
cln_rate  = calc_compression_rate(NG, G_CLN)

final_nodes = G_CLN_ADMET.number_of_nodes()
final_edges = G_CLN_ADMET.number_of_edges()
final_size  = get_size_kb(G_CLN_ADMET)
final_rate  = calc_compression_rate(NG, G_CLN_ADMET)

# -----------------------------
# 표(DataFrame)
# -----------------------------
df = pd.DataFrame({
    "방식": ["Original", "ENN", "CLN", "CLN-ADMET"],
    "Nodes": [original_nodes, enn_nodes, cln_nodes, final_nodes],
    "Edges": [original_edges, enn_edges, cln_edges, final_edges],
    "Size(KB)": [original_size, enn_size, cln_size, final_size],
    "압축율(%)": ["-", f"{enn_rate}%", f"{cln_rate}%", f"{final_rate}%"]
})

print("\n📊 Step 8 : 최종 정리 성능 요약")
display(df)

# -----------------------------
# Summary (이미지 느낌 그대로)
# -----------------------------
print("\n📝 Summary")
print(f" - ENN : 정리율 {enn_rate}%  → 기본 구조를 자연스럽게 정리하는 단계")
print(f" - CLN : 정리율 {cln_rate}%  → 문맥 단위로 구조를 다듬어 정보 흐름 개선")
print(f" - CLN-ADMET : 압축율 {final_rate}% → 최종적으로 활용하기 가장 적합한 균형 상태")


📊 Step 8 : 최종 정리 성능 요약


,방식,Nodes,Edges,Size(KB),압축율(%)
0,Original,21040,38449,1384.16,-
1,ENN,17894,30862,1111.03,19.73%
2,CLN,7996,16423,591.23,57.29%
3,CLN-ADMET,6731,12578,452.81,67.29%



📝 Summary
 - ENN : 정리율 19.73%  → 기본 구조를 자연스럽게 정리하는 단계
 - CLN : 정리율 57.29%  → 문맥 단위로 구조를 다듬어 정보 흐름 개선
 - CLN-ADMET : 압축율 67.29% → 최종적으로 활용하기 가장 적합한 균형 상태
